In [ ]:
#필요라이브러리 import
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import pyedflib
import matplotlib.pyplot as plt
import tensorflow as tf
import sys, math ,os
import pickle
from pywt import wavedec
import pywt
import torch

In [ ]:
#데이터셋 디렉터리 및 파일 수 확인

dir_list = [os.getcwd() + "/dataset/2_Test",os.getcwd() + "/dataset/1_Train+Val"]
for i, dataset_dir in enumerate(dir_list):
    file_list = os.listdir(dataset_dir)
    file_list_py = [file for file in file_list if file.endswith(".edf")]
    file_list_py.sort()
    print(f"{dir_list[i]} 파일 개수 : {len(file_list_py)}")

In [ ]:
#데이터셋 라벨링 OSA : 1 / Normal : 0

file_labels = dict()
for filename in os.listdir("/home/maic-player/dataset/1_Train+Val"):
    file_labels.update({filename:1 if "OSA" in filename else 0})
# print(file_labels)

In [ ]:
# 데이터셋 duration min,max,average 확인

duration_list = list()
for f_name in file_list_py:
    edf = pyedflib.EdfReader(dataset_dir + "/"+  f_name)
    duration_list += [edf.getFileDuration()]
    edf.close()
plt.plot(duration_list,label=f_name)
plt.show()
print(
    f'''
    최소 시간: {min(duration_list)/3600}시간({min(duration_list)})
    최대 시간: {max(duration_list)/3600}시간({max(duration_list)})
    평균 시간: {np.mean(duration_list)/3600}시간({np.mean(duration_list)})''')

In [ ]:
#edf 내부 label 확인
labels = edf.getSignalLabels()
print(len(labels))
print(labels)

In [ ]:
dataset_dir + "/"+  file_list_py[80]

In [ ]:
edf_File = file_list_py[40]
edf = pyedflib.EdfReader(f"{os.getcwd()}/dataset/1_Train+Val/{edf_File}")
print(edf_File)

In [ ]:
print("Duaration:"+str(edf.getFileDuration())) # 파일 Duration 확인
print("Freq.:"+str(edf.getSampleFrequencies())) # 파일 Frequen
print("N-Sample(=Freq x Duaration):"+str(edf.getNSamples()))
print("Date:"+str(edf.getStartdatetime()))

In [ ]:
edf.readSignal(0).size

1000개 -> 10초간 측정된 데이터

최소 시간: 1.25시간(4500)

최대 시간: 12.25시간(44100)

평균 시간: 6.291944444444445시간(22651.0)

In [ ]:
# xyz plot
ten_sec=1000 # 멜스펙토그램 기준 1000 -> 10초
for i in range(0,3):
    plt.plot(edf.readSignal(i)[10*ten_sec:20*ten_sec], # 100초 ~ 200초 사이 signal 데이터 읽어 보기
             label=labels[i])
plt.legend()
plt.show()

In [ ]:
# ECG plot
index = 3 # index 3번 ECG 데이터
plt.plot(edf.readSignal(index)[10*ten_sec:20*ten_sec], # 100초 ~ 200초 사이 signal 데이터 읽어 보기
         label=labels[index])
plt.legend()
plt.show()

In [ ]:
# mel freq plot
for i in range(4, 24): # index 4~24 까지 멜스팩토그램 20개 채널
    plt.plot(edf.readSignal(i)[10*ten_sec:60*ten_sec], # 100초 ~ 200초 사이 signal 데이터 읽어 보기
             label=labels[i])
# plt.legend()
plt.show()

In [ ]:
print("Anotation:"+str(edf.read_annotation()))
print("Technician:"+str(edf.getTechnician()))
print("Header:"+str(edf.getHeader()))

In [ ]:
print("SIgnal-Header(CH-0):"+str(edf.getSignalHeader(0))+"¥n")

In [ ]:
print(edf.file_info())
edf.close()

In [ ]:
# torch gpu 환경 테스트
print(torch.backends.cudnn.is_available())
print(torch.backends.cudnn.version())
print(torch.backends.cudnn.enabled)
print(torch.cuda.get_device_properties("cuda"))

In [ ]:
# ecg 심전도 데이터 노이즈 처리(레퍼런스 : https://www.kaggle.com/code/nelsonsharma/ecg-02-ecg-signal-pre-processing)
def denoise_signal(X, dwt_transform, dlevels, cutoff_low, cutoff_high):
    coeffs = wavedec(X, dwt_transform, level=dlevels)   # wavelet transform 'bior4.4'
    # scale 0 to cutoff_low
    for ca in range(0,cutoff_low):
        coeffs[ca]=np.multiply(coeffs[ca],[0.0])
    # scale cutoff_high to end
    for ca in range(cutoff_high, len(coeffs)):
        coeffs[ca]=np.multiply(coeffs[ca],[0.0])
    Y = pywt.waverec(coeffs, dwt_transform) # inverse wavelet transform
    return Y

# edf 데이터 호출
def get_edf_data(default_path:str):
    data_dict = {
     "mov":list(),
     "ecg":list(),
     "mel":list(),
     "label":list()
    }
    file_list_py = os.listdir(default_path)
    for flp in tqdm(file_list_py):
        if flp.endswith(".edf"):
            edf = pyedflib.EdfReader(f"{default_path}/{flp}") #edf 데이터 읽기
            #print(file_list_py[k+71]
            X_axis = edf.readSignal(0) #액티그래피 x축 데이터 변수저장
            Y_axis = edf.readSignal(1) #액티그래피 y축 데이터 변수저장
            Z_axis = edf.readSignal(2) #액티그래피 z축 데이터 변수저장
            Mov_volume_list = [math.sqrt(math.pow(X_axis[j], 2) + math.pow(Y_axis[j], 2) + math.pow(Z_axis[j], 2)) \
                               for j in range(len(X_axis))] #x,y,z 데이터 1차원의 움직임 값으로 변환공식
            freq = edf.getSampleFrequencies()[0]
            xyz_n_sample = edf.getNSamples()[0]
            xyz = list()
            for n in range(freq, xyz_n_sample, freq):
                xyz += [Mov_volume_list[n:n+freq]]

            ECG_axis = edf.readSignal(3) #ECG 원천데이터 변수저장
            ECG_axis = denoise_signal(ECG_axis,'bior4.4', 9 , 1 , 7) #ECG 원천데이터 denoise 적용
            freq = edf.getSampleFrequencies()[3]
            ecg_n_sample = edf.getNSamples()[3]
            ecg = list()
            for n in range(freq, ecg_n_sample, freq):
                ecg += [ECG_axis[n:n+freq]]

            MEL_axis = np.array([edf.readSignal(index) for index in range(4,9)]) #코골이 주파수 대역 멜스펙토그램 데이터 저장
            freq = edf.getSampleFrequencies()[4]
            mel_n_sample = edf.getNSamples()[4]
            mel = list()
            for n in range(freq, mel_n_sample, freq):
                mel += [MEL_axis[:,n:n+freq]]
            freq_list = list()
            freq_list = [[np.array(m)[:,idex].sum() for idex in range(freq)] for m in np.array(mel)]
            # for i in range(freq):
                # pcaed += [pca.fit_transform([m[:,i] for m in mel]).tolist()]
            edf.close()

            Mov_volume_list = np.array(xyz) #액티그래피 1차원 움직임 List
            ECG_volume_list = np.array(ecg) #Denoise 된 ECG 데이터 List
            MEL_volume_list = np.array(freq_list) #멜 스펙토그램 List
            data_dict["mov"] += [[Mov_volume_list[i,:].sum() for i in range(len(Mov_volume_list))]] #1차원 움직임 dict
            data_dict["ecg"] += [[ECG_volume_list[i,:].sum() for i in range(len(Mov_volume_list))]] #Denoise ECG dict
            data_dict["mel"] += [[MEL_volume_list[i,:].sum() for i in range(len(Mov_volume_list))]] #멜스펙토그램 dict
            data_dict["label"] += [1 if "OSA" in flp else 0] #OSA : 1 / Normal : 0 라벨 Dict
    return data_dict

In [ ]:
# 1.Train+Val 디렉토리 edf 파일 읽어오기
df = get_edf_data(default_path = os.getcwd() + "/dataset/1_Train+Val")
# 2.Test 디렉토리 edf 파일 읽어오기
Test = get_edf_data(default_path = os.getcwd() + "/dataset/2_Test")

# 새로운 dataframe 저장을 위한 dict형의 변수
data_dict = {
 "mov":list(),
 "ecg":list(),
 "mel":list(),
 "label":list()
}
dur = 450
# Train 데이터만 최소 duration 450 기준으로 split하여 301개 데이터에서 1888개로 확장
for index, row in tqdm(df.iterrows(),
                       total = len(df)):
    start = 0
    # 0에서 부터 duration에 해당하는 데이터까지 반복하여 새로운 데이터로 가져옴
    while len(row['mov'][start:start+dur]) >= dur:
        data_dict["mov"] += [row['mov'][start:start+dur]]
        data_dict["ecg"] += [row['ecg'][start:start+dur]]
        data_dict["mel"] += [row['mel'][start:start+dur]]
        data_dict["label"] += [row['label']]
        start += dur
df = pd.DataFrame(data_dict)

# Test 데이터의 경우 450까지만 가져오도록 함
for index, row in tqdm(Test.iterrows(),
                       total = len(df)):
    start = 0
    row['mov'] = row['mov'][start:start+dur]
    row['ecg'] = row['ecg'][start:start+dur]
    row['mel'] = row['mel'][start:start+dur]

# Train 80 : Valid 20의 비율로 data spilte
Train, Valid, _, _ = train_test_split(df,
                                      df.label,
                                      stratify=df.label,
                                      test_size=0.2,
                                      random_state=42)

# Valid 데이터를 통한 정확한 성능 측정을 위해 Valid 데이터는 OSA와 Normal 비율을 50:50으로 맞춤
Valid = pd.concat([Valid[Valid.label==1].sample(Valid.label.value_counts()[0]),
                   Valid[Valid.label==0]], ignore_index=True)

# 빠르게 데이터를 읽어오기 위해 DataFrame 형태 그대로 저장
with open("train.pk","wb") as f:
    pickle.dump(Train, f)
    f.close()
with open("valid.pk","wb") as f:
    pickle.dump(Valid, f)
    f.close()
with open("test.pk","wb") as f:
    pickle.dump(Test, f)
    f.close()

In [ ]:
%%writefile model.py
from torch import nn

# mov, ecg, mel 세 개의 데이터를 읽어와서 처리하는 CNN 구조의 모델 정의
class Model(nn.Module):
    def __init__(self,
                 device="cpu"):
        super().__init__()
        self.device=device
        self.cnn = nn.Sequential(
                    nn.Conv2d(in_channels=1,
                              out_channels=10,
                              stride=3,
                              kernel_size=(3, 3)),
                    nn.Dropout(p=0.3),
                    nn.Conv2d(in_channels=10,
                              out_channels=20,
                              stride=2,
                              kernel_size=(1, 5)),
                    nn.Dropout(p=0.3),
                    nn.Conv2d(in_channels=20,
                              out_channels=20,
                              stride=2,
                              kernel_size=(1, 3)),
                    nn.Dropout(p=0.3),
                    nn.Conv2d(in_channels=20,
                              out_channels=10,
                              stride=3,
                              kernel_size=(1, 2)),
                    nn.ReLU(),
                    nn.BatchNorm2d(10),
                    nn.MaxPool2d((1, 1),
                                 stride=2),
                    nn.Dropout(p=0.3),
                    nn.Conv2d(in_channels=10,
                              out_channels=1,
                              stride=2,
                              kernel_size=(1, 1)),
                    nn.Flatten()
                    )
        self.mlp = nn.Sequential(
            nn.Linear(3, 2),
            nn.Linear(2, 2),
            nn.Sigmoid(),
        )

    def forward(self,
                inputs,
                return_dict=False):
        outputs = self.cnn(inputs)
        return  self.mlp(outputs)

In [ ]:
%%writefile trainer.py
import argparse
import logging
import os
import gc
import numpy as np
import pandas as pd
import time
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import (TQDMProgressBar,
                                         StochasticWeightAveraging,
                                         DeviceStatsMonitor)
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW
from torchmetrics.text.bleu import BLEUScore
from tqdm.auto import tqdm
import pyedflib
import datetime
from model import Model
import pickle
import torch.distributed as dist

# Use address of one of the machines

# 주피터 환경에서 gpu 사용하는 경우 kernel에서 계속 gpu 메모리를 점유하고 반환하지 않기 때문에,
# 학습에 대한 다양한 접근을 해보기 어렵기 때문에 수행 종료후 메모리 반환을 하는
# .py 수행으로 학습을 진행
# cls level에서 하이퍼파라미터 입력을 가능하게 하기 위한 argparse
parser = argparse.ArgumentParser(description='OSA TRAINER')
# 기본 checkpoint 경로 지정
parser.add_argument('--checkpoint_path',
                    type=str,
                    help='checkpoint path')
# torch에서 gpu 인식되는 경우 gpu 사용. 아닌 경우 cpu로 연산
device = "cuda" if torch.cuda.is_available else "cpu"

# 로거 지정
logger = logging.getLogger()
# INFO 수준의 log도 출력
logger.setLevel(logging.INFO)


class CustomDataset(Dataset):
    def __init__(self,
                 is_train:bool = True,
                 device:str = "cpu"):
        self.device = device
        if is_train:
            # train 모드일 때 train 데이터 프레임 가져옴
            with open("train.pk", "rb")as f:
                self.dataset = pickle.load(f)
        else:
            # valid 모드일 때 valid 데이터 프레임 가져옴
            with open("valid.pk", "rb")as f:
                self.dataset = pickle.load(f)


    def __getitem__(self,
                    index: int):
        # item iteration 마다 수행하게 됨
        example = self.dataset.iloc[index,:]
        # list 형 데이터 numpy arry float 32 타입으로 수정
        data = np.array(example.data, dtype=np.float32)
        # int 형 라벨 numpy arry long 타입으로 로 수정
        label = np.array(example.label, dtype=np.int_)
        # 모델에 바로 입력하기 위해 torch tensor 형태로 반환
        return {
            "data":torch.tensor(data),
            "label":torch.tensor(label)
        }

    def __len__(self) -> int:
        # dataset에 대한 길이 반환
        return len(self.dataset)


class ArgsBase():
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        # 기본 train data 경로
        parser.add_argument('--train_file',
                            type=str,
                            default='train.pk',
                            help='train file')
        # 기본 valid data 경로
        parser.add_argument('--test_file',
                            type=str,
                            default='valid.pk',
                            help='test file')

        return parser


class OSADataModule(pl.LightningDataModule):
    def __init__(self,
                 batch_size=64,
                 num_workers=8):
        super().__init__()
        # argument에서 입력하는 batch size를 받아옴. 기본은 64
        self.batch_size = batch_size
        # argument에서 입력하는 dataloader workers 받아옴. 기본은 8
        self.num_workers = num_workers
        # 사용하는 gpu 노드 당 데이터를 load
        self.prepare_data_pre_node = True
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        parser.add_argument('--num_workers',
                            type=int,
                            default=8,
                            help='num of worker for dataloader')
        return parser

    def setup(self, stage):
        # 모듈 수행 시 train 데이터에 대한 dataset 객체 생성
        self.train = CustomDataset(is_train=True,
                                   device=device)
        # 모듈 수행 시 validaion 데이터에 대한 dataset 객체 생성
        self.test = CustomDataset(is_train=False,
                                  device=device)

    # train data loader 함수
    def train_dataloader(self):
        return DataLoader(self.train,
                          batch_size=self.batch_size, # 한번에 batch size만큼 데이터를 load
                          num_workers=self.num_workers, # data loader 수행하는 process 수
                          pin_memory=True, # 메모리에 dataloaer 값 미리 고정시킴
                          shuffle=True) # 데이터가져올 때 shuffle 수행
    # validation data loader 함수
    def val_dataloader(self):
        return DataLoader(self.test,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          pin_memory=True,
                          shuffle=False)
    # test data loader 함수
    def test_dataloader(self):
        return DataLoader(self.test,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          pin_memory=True,
                          shuffle=False)


class Base(pl.LightningModule):
    def __init__(self, hparams, **kwargs) -> None:
        super(Base, self).__init__()
        self.save_hyperparameters(hparams)

    @staticmethod
    def add_model_specific_args(parent_parser):
        # add model specific args
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        # 기본 batch size 파라미터 지정
        parser.add_argument('--batch-size',
                            type=int,
                            default=8,
                            help='batch size for training (default: 96)')
        # 기본 learning rate 파라미터 지정
        parser.add_argument('--lr',
                            type=float,
                            default=5e-7,
                            help='The initial learning rate')
        # 기본 warmup ratio 지정
        parser.add_argument('--warmup_ratio',
                            type=float,
                            default=0.2,
                            help='warmup ratio')

        return parser

    def configure_optimizers(self):
        # Optimizer에서 Backward 수행할 Parameter 지정
        param_optimizer = list(self.model.named_parameters())
        # AdamW optimizer에서 no_decay로 지정되어야 하는 항목들
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(
                nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        # AdamW Optimizer 선언
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr = self.hparams.lr)
        # Optimizing process 수
        num_workers = (self.hparams.gpus if self.hparams.gpus is not None else 1) * (self.hparams.num_nodes if self.hparams.num_nodes is not None else 1)
        # Dataloader에서 읽어오는 dataset 길이
        data_len = len(self.trainer._data_connector._train_dataloader_source.dataloader().dataset)
        logging.info(f'number of workers {num_workers}, data length {data_len}')
        # Warmup 및 CosineAnnealing에 따른 Learning rate 조절을 위한 전체 Training Step 계산
        num_train_steps = int(data_len / (self.hparams.batch_size * num_workers) * self.hparams.max_epochs)
        logging.info(f'num_train_steps : {num_train_steps}')
        # Warmup Ratio에 따라 warmup을 수행할 step 수 계산
        num_warmup_steps = int(num_train_steps * self.hparams.warmup_ratio)
        logging.info(f'num_warmup_steps : {num_warmup_steps}')
        # CosineAnnealing Learning Rate Scheduler 선언
        scheduler = CosineAnnealingLR(optimizer=optimizer,
                                      T_max=num_train_steps,
                                      eta_min=5e-8)
        lr_scheduler = {'scheduler': scheduler,
                        'monitor': 'train_loss',
                        'interval': 'step',
                        'frequency': 1}
        return [optimizer], [lr_scheduler]


class MyTrainer(Base):
    def __init__(self, hparams, **kwargs):
        super(MyTrainer, self).__init__(hparams, **kwargs)
        # 모델 정의
        self.model = Model()
        # 이진 분류 output 2개에 대한 Loss Function
        self.loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor([.79,.67]),
                                                 reduction="mean")
        # 성능 측정을 위한 F1 Metric과 Accuracy
        self.metric = BinaryF1Score()
        self.acc = BinaryAccuracy()


    def forward(self, inputs):
        # input에 대한 Model 연산
        outputs = self.model(inputs=inputs['data'])
        # ouput의 예측값에 대한 argmax
        preds = torch.argmax(outputs, dim=1)
        # label 과 preds 사이의 loss 계산
        loss = self.loss_fn(outputs, inputs['label'])
        return {
            "loss": loss,
            "logits": outputs,
            "preds": preds
        }


    def training_step(self, batch, batch_idx):
        # batch 데이터 trainer에 넘기기
        outs = self(batch)
        # 예측값에 대한 f1 metric 계산
        metric = self.metric(outs["preds"], batch['label'])
        # 예측값에 대한 accuracy 계산
        acc = self.acc(outs["preds"], batch['label'])
        # loss, metric, acc log로 출력
        self.log('train_loss', outs["loss"], prog_bar=True, on_step=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True, on_step=True, on_epoch=True, logger=True)
        self.log('train_f1', metric, prog_bar=True, on_step=True, on_epoch=True, logger=True)
        # loss Backward를 위한 loss 값 반환
        return outs["loss"]

    def validation_step(self, batch, batch_idx):
        # batch 데이터 trainer에 넘기기
        outs = self(batch)
        # 예측값에 대한 f1 metric 계산
        metric = self.metric(outs["preds"], batch['label'])
        # 예측값에 대한 accuracy 계산
        acc = self.acc(outs["preds"], batch['label'])
        # loss, metric, acc log로 출력
        self.log('val_loss', outs["loss"], prog_bar=True, on_step=False, on_epoch=True, logger=True)
        self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
        self.log('val_f1', metric,prog_bar=True, on_step=False, on_epoch=True, logger=True)


if __name__ == '__main__':
    # Pytorch GPU 사용을 위한 환경변수 수정
    os.environ["NCCL_IB_DISABLE"] = "1"
    os.environ["NCCL_P2P_DISABLE"] = "1"
    os.environ["NCCL_DEBUG"] = "WARNNING"
    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
    os.environ["WORLD_SIZE"] = "3"
    os.environ["LOCAL_RANK"] = "0"
    # torch 랜덤 시드 값 고정
    torch.cuda.manual_seed_all(seed=42)
    # 기본 Trainer Argument 값 지정 및 추가
    parser = Base.add_model_specific_args(parser)
    parser = ArgsBase.add_model_specific_args(parser)
    parser = OSADataModule.add_model_specific_args(parser)
    # Trainer에서 Validation 데이터셋 F1 score 모니터링을 통한 Early Stopping callback 추가
    parser = pl.Trainer(callbacks=[EarlyStopping(monitor="val_f1",
                                                 stopping_threshold=1e-4,
                                                 min_delta=0.00,
                                                 patience=10,
                                                 divergence_threshold=9.0),
                                   ]).add_argparse_args(parser)
    # argument 읽어오기
    args = parser.parse_args()
    # logging.info(args)
    model = MyTrainer(args)
    # Data Loader 모듈에 Batch size와 loader process 수 지정
    dm = OSADataModule(batch_size=args.batch_size,
                       num_workers=args.num_workers)
    # Validation 데이터 셋 F1 score 기준으로 best 모델 저장
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_f1',
                                                       dirpath=args.default_root_dir,
                                                       filename='best-checkpoint',
                                                       verbose=True,
                                                       save_last=True,
                                                       mode='max',
                                                       save_top_k=1,)
    # 학습 모니터링을 위한 TensorBoard logger 선언
    tb_logger = pl_loggers.TensorBoardLogger(os.path.join(args.default_root_dir,'tb_logs'),
                                             log_graph=False,)
    # 학습 중 모델의 학습에 대한 모니터링을 위한 WandB 플랫폼 logger 연결
    # wandb_logger = pl_loggers.WandbLogger(name="MAIC",
    #                                       project="Sleep AI Challenge ver3",
    #                                       log_model="all")
    # LearningRate 모니터링 callback 추가
    lr_logger = pl.callbacks.LearningRateMonitor()
    # Trainer에 받아온 Argument값 넣기
    trainer = pl.Trainer.from_argparse_args(args,
                                            logger=[tb_logger,
                                                    # wandb_logger # training 중 모니터링을 위한 wandb 플랫폼 로그는 제출시 주석 처리 하였습니다.
                                                   ],
                                            callbacks=[checkpoint_callback,
                                                       lr_logger])
    # Trainer 모델 학습 수행
    trainer.fit(model, dm, )
    # 학습 종료 시점의 Model 추가 저장
    save_path = f'logs/osa{datetime.datetime.now().strftime("%Y.%m.%d_%H.%M.%S")}.pt'
    torch.save(model.model.state_dict(), save_path)
    print(f'### torch  model.pth has saved at {save_path} ###')

In [ ]:
# 주피터 커널이 GPU를 점유하고 있어 python 파일로 실행할 수 있게 함
!python \
    trainer.py\
        --gradient_clip_val 1.0\
        --max_epochs 50\
        --lr 5e-3\
        --batch-size 16\
        --precision 32\
        --default_root_dir logs\
        --accelerator gpu\
        --devices -1\
        --strategy ddp\
        --num_nodes 1\
        --num_workers 1\
        --auto_select_gpus true\
        --replace_sampler_ddp true\
        --sync_batchnorm true\
        --move_metrics_to_cpu false\
        --profiler simple\
        --enable_model_summary true\
        --amp_backend native\
        --fast_dev_run false

제출 코드

In [ ]:
from model import Model
from trainer import *
import torch
import easydict
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import pickle

args = easydict.EasyDict({
    "gradient_clip_val": 1.0,
    "max_epochs": 50,
    "lr": 5e-4,
    "batch_size": 64,
    "precision": 32,
    "default_root_dir": "logs",
    "num_workers":0,
})
# Trainer 객체 선언을 위한 기본 argument 값 넣기
model = MyTrainer(args)
# 학습한 Trainer에 모델 state_dict 값 로드
model.load_state_dict(torch.load("BACKUP/logs/best-checkpoint-v50.ckpt")["state_dict"])

In [ ]:
# 테스트 Dataframe pickle 파일 로드
with open("test.pk","rb") as f:
    submit_df = pickle.load(f)
    f.close()

# 제출용 답안 작성 파일명 Test_Answer_Form.csv 파일을 읽어서 모델을 통한 정답 마킹
a = pd.read_csv("Test_Answer_Form.csv", names=["name"])
for index,row in tqdm(submit_df.iterrows(),
                      total=len(submit_df),
                      desc="marking labels...."):
    # 테스트 데이터 값을 입력 가능한 torch.tensor의 형태로 변환
    inputs = torch.tensor(np.array([row["data"]], dtype=np.float32))
    # trainer의 모델을 통한 output 연산
    output = model.model.forward(inputs)
    # 25, 50, 75 인덱스에 대해 결과 연산에 문제가 없는지 확인
    if index in [25,50,75]:
        print(output)
    # [1,2] 사이즈의 tensor에서 argmax 연산을 통해 binary class 매핑
    label="OSA" if output.argmax(1) else "Normal"
    a.loc[index,"label"] = label

# 제출용 csv 파일로 저장
a.to_csv("Submissions.csv",
         header=None,
         index=False)